# 11-HUBO: Higher order unconstraint binary optimization

If you want to handle higher order model as follows:

$$
H=\sum_{i} h_{i} \sigma_{i}+\sum_{i<j} J_{i j} \sigma_{i} \sigma_{j}+\sum_{i, j, k} K_{i, j, k} \sigma_{i} \sigma_{j} \sigma_{k} \cdots 
\\
\sigma_i \in \{-1, 1\}, i=1,\cdots N
$$



## HUBOの定義

In [1]:
import openjij as oj

# Only SASampler can handle HUBO.
sampler = oj.SASampler()

# define polynomial
polynomial = {(0,): -1, (0,1): -1, (0,1,2): 1}

## QUBO変換による解法

In [4]:
import dimod

# Polynomial相互作用、ペナルティの大きさ、変数のタイプを指定して対応するquadraticモデルを生成する。
bqm = dimod.make_quadratic(poly=polynomial, strength=5.0, vartype="SPIN")
print('0次の項:', bqm.offset)
print('1次の項:', dict(bqm.linear))    # bqm.linearはpythonのdictに変換して表示する。
print('2次の項:', dict(bqm.quadratic)) # bqm.quadraticもpythonのdictに変換して表示する。

0次の項: 10.0
1次の項: {0: -3.5, '0*1': -3.5, 2: 0.0, 1: -2.5, 'aux0,1': -5.0}
2次の項: {('0*1', 0): 2.5, (2, '0*1'): 1.0, (1, 0): 2.5, (1, '0*1'): 2.5, ('aux0,1', 0): 5.0, ('aux0,1', '0*1'): 5.0, ('aux0,1', 1): 5.0}


In [5]:
# インデックスを0始まりの整数に変換する関数
def relabel_variables_as_integers(dimod_bqm):
    mapping = {}
    variables = list(dimod_bqm.variables)
    count = 0
    for key in variables:
        mapping[key] = count
        count += 1
    linear    = {mapping[k]:v for k,v in dimod_bqm.linear.items()}
    quadratic = {(mapping[k[0]], mapping[k[1]]):v for k,v in dimod_bqm.quadratic.items()}
    return dimod.BinaryQuadraticModel(linear, quadratic, dimod_bqm.offset, dimod_bqm.vartype), mapping


bqm_relabeled, mapping = relabel_variables_as_integers(bqm) # インデックスを0始まりに変換する。

print('0次の項:', bqm_relabeled.offset)
print('1次の項:', dict(bqm_relabeled.linear))    # bqm.linearはpythonのdictに変換して表示する。
print('2次の項:', dict(bqm_relabeled.quadratic)) # bqm.quadraticもpythonのdictに変換して表示する。
print('変数の対応関係:', mapping) # Relabelしたあ後のインデックスと元のインデックスの対応関係を表示する。

0次の項: 10.0
1次の項: {1: -3.5, 0: -3.5, 2: 0.0, 3: -2.5, 4: -5.0}
2次の項: {(0, 1): 2.5, (2, 1): 1.0, (3, 1): 2.5, (3, 0): 2.5, (4, 1): 5.0, (4, 0): 5.0, (4, 3): 5.0}
変数の対応関係: {0: 0, '0*1': 1, 2: 2, 1: 3, 'aux0,1': 4}


In [21]:
# dimodのbqm_relabeledはOpenJijのBinaryQuadraticModelに変換する。
bqm_oj = oj.BinaryQuadraticModel(dict(bqm_relabeled.linear), dict(bqm_relabeled.quadratic), bqm_relabeled.offset, vartype="SPIN")

#　ここまでの前処理をした後に、sampleメソッドに投げることができる。
response = sampler.sample(bqm_oj)
print(response)

hubo_configuration = {i: response.record[0][0][mapping[i]] for i in range(len(polynomial))}
print('対応するHUBOの解:', hubo_configuration)
print('対応するHUBOの解のエネルギー:', dimod.BinaryPolynomial(polynomial, "SPIN").energy(hubo_configuration))

   0  1  2  3  4 energy num_oc.
0 +1 +1 -1 +1 -1   -3.0       1
['SPIN', 1 rows, 1 samples, 5 variables]
対応するHUBOの解: {0: 1, 1: 1, 2: -1}
対応するHUBOの解のエネルギー: -3.0


In [22]:
# 元のHUBOの最適解の確認
sampleset = dimod.ExactPolySolver().sample_hising(h = {}, J = polynomial)
print('最適解:',sampleset.first.sample)
print('対応するエネルギー:',sampleset.first.energy)

最適解: {0: 1, 1: 1, 2: -1}
対応するエネルギー: -3.0


### QUBO変換による解法の問題点
1. ペナルティの大きさ(strength)の適切な値が不明瞭
2. QUBO変換した際に変数、相互作用の数が増えてしまい、変換時間もかかってしまう

## HUBOの直接解法

In [23]:
# make HUBO
response = sampler.sample_hubo(polynomial, vartype="SPIN")
print(response)

   0  1  2 energy num_oc.
0 +1 +1 -1   -3.0       1
['SPIN', 1 rows, 1 samples, 3 variables]


In [24]:
# 辞書のkeyは数値以外も扱うことができる。
# 相互作用を定義
polynomial_ = {('a',): -1, ('a','b'): -1, ('a','b','c'): 1}

# HUBOを作り、sample_huboメソッドに投げる。
response = sampler.sample_hubo(polynomial_, vartype="SPIN")
# 最適化の結果を取得
print(response)

   a  b  c energy num_oc.
0 +1 +1 -1   -3.0       1
['SPIN', 1 rows, 1 samples, 3 variables]
